# LabCamp Training AI Agent with RAG - FineTuning - Dataset Preparation

## Descrizione

Questo notebook Python è stato sviluppato per preparare un dataset per il fine-tuning di un modello LLM si OpenAI come **GPT-4o-mini**.

Il modello è viene addestrato con esempi di conversazione pertinenti.

L'intero processo include:

- Estrazione del testo da un file PDF utilizzando **Azure AI Document Intelligence**
- Generazione automatica di esempi di conversazione basati sul contenuto estratto
- Validazione del dataset e stima dei costi di addestramento su OpenAI

## Installazione delle dipendenze

Lo script richiede alcune librerie specifiche. È possibile installarle eseguendo: pip install ...

## Prerequisiti

1. Rinominare il file in doc.pdf o modificare l'opportuna variabile nel codice

In [1]:
pip install --upgrade --quiet  langchain langchain-community azure-ai-documentintelligence openai tiktoken python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import dotenv, os
dotenv.load_dotenv(override=True)

endpoint = "https://labcamp-document-intelligence.cognitiveservices.azure.com/"
documentPath = 'doc.pdf'

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
DOC_INTELLIGENCE_KEY = os.getenv("DOC_INTELLIGENCE_KEY")

# Lettura documento

Lettura del documento che fungirà da knowledge source/contesto per creare il dataset in modo sintetico.

Il documento verrà letto tramite l'ausilio del tool Azure AI Document Intelligence.

In [ ]:
# Import libraries
import os
from langchain_community.document_loaders import AzureAIDocumentIntelligenceLoader

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs if not isinstance(doc, dict) and hasattr(doc, 'page_content'))

# Create Document Intelligence client
loader = AzureAIDocumentIntelligenceLoader(
    api_key=DOC_INTELLIGENCE_KEY,
    api_endpoint=endpoint,
    api_version='2024-02-29-preview',
    file_path=documentPath,
    api_model='prebuilt-layout',
    mode='markdown'
)
pages = loader.load()
full_text = format_docs(pages)
file_metadata = [page.metadata for page in pages]
print('File read successfully with Azure Document Intelligence.')
print(f"Here's the full content in format {full_text}:\n")

# Save to file .txt
context_file_path = 'context.txt'
with open(context_file_path, "w", encoding='utf-8') as file:
  file.write(full_text)

print(f"Markdown file saved as {context_file_path}")

File read successfully with Azure Document Intelligence.
Markdown file saved as context.txt


# Prompt creazione dataset

In [10]:
system_prompt = "L'assistente è un chatbot esperto del Festival di Sanremo che fornisce risposte formali e dettagliate."

prompt = """
ROLE
Sei un generatore di esempi di training per fine tuning esperto di sanremo.
Devi creare {{num_examples}} esempi con un formato simile a ESEMPI
Non mostrare l'output. Devi generare un file di testo con tutti i risultati

TASK
Gli esempi devono essere formali, devi essere educato e gentile.
In allegato hai le informazioni che devi utilizzare per generare gli esempi.

Devi coprire tutti i temi utilizzati nel documento.
Cerca di fare domande varie, anche in forma diversa.

====ESEMPI
{"messages": [{"role": "system", "content": "L'assistente è un chatbot esperto del Festival di Sanremo che fornisce risposte formali e dettagliate."}, {"role": "user", "content": "Chi ha vinto il Festival di Sanremo 2024?"}, {"role": "assistant", "content": "Il vincitore del Festival di Sanremo 2024 è stato Angelina Mango con il brano 'La noia'."}]}
{"messages": [{"role": "system", "content": "L'assistente è un chatbot esperto del Festival di Sanremo che fornisce risposte formali e dettagliate."}, {"role": "user", "content": "Quale artista ha ricevuto il Premio della Critica nel 2024?"}, {"role": "assistant", "content": "Il Premio della Critica del Festival di Sanremo 2024 è stato assegnato a Loredana Bertè."}]}
{"messages": [{"role": "system", "content": "L'assistente è un chatbot esperto del Festival di Sanremo che fornisce risposte formali e dettagliate."}, {"role": "user", "content": "Quanti partecipanti hanno preso parte al Festival di Sanremo 2024?"}, {"role": "assistant", "content": "Al Festival di Sanremo 2024 hanno partecipato 30 artisti in gara."}]}
===
<CONTEXT>
{{context}}
</CONTEXT>
"""

# Creazione dataset

Tramite le opportune variabili è possibile regolare il numero di esempi da creare per il dataset.

Lo step produrrà un file `dataset.jsonl`

In [ ]:
import asyncio
import json
from openai import AsyncOpenAI

input_file = "context.txt"
output_file = "dataset.jsonl"

total_examples = 5  # Numero totale di esempi da generare
num_requests   = 5    # Numero di richieste parallele (ciascuna genererà una parte degli esempi)

openai_model="gpt-4o-mini"

client = AsyncOpenAI(api_key=OPENAI_API_KEY)

def read_context(file_path):
    """Legge il contenuto del file di contesto in UTF-8."""
    with open(file_path, "r", encoding="utf-8") as f:
        return f.read()

async def call_openai_api(prompt_text):
    """
    Effettua una richiesta all'API di OpenAI utilizzando il prompt fornito.
    Viene inviato un messaggio di sistema seguito dal messaggio utente contenente il prompt.
    """
    try:
        chat_completion = await client.chat.completions.create(
            model=openai_model,
            temperature=1.0,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt_text}
            ]
        )
        result = chat_completion.choices[0].message.content.strip()
        # Rimuove eventuali righe vuote
        result = "\n".join([line for line in result.splitlines() if line.strip() != ""])
        print(result)
        print("="*50)
        return result
    except Exception as e:
        print(f"❌ Errore nella richiesta API: {e}")
        return ""

async def process_batches(prompt_template, total_examples, num_requests):
    """
    Divide il numero totale di esempi da generare tra 'num_requests' richieste in parallelo.
    Per ciascuna richiesta viene sostituito il placeholder {{num_examples}} con il numero di esempi
    da generare in quel batch.
    """
    tasks = []
    base = total_examples // num_requests
    remainder = total_examples % num_requests
    # Distribuisce il resto (se presente) fra le prime richieste
    batch_sizes = [base + 1 if i < remainder else base for i in range(num_requests)]

    for batch_size in batch_sizes:
        prompt_batch = prompt_template.replace("{{num_examples}}", str(batch_size))
        tasks.append(call_openai_api(prompt_batch))

    responses = await asyncio.gather(*tasks)
    return responses

def parse_responses(responses):
    """
    Parsa le risposte ottenute, aspettandosi che ogni riga contenga un oggetto JSON.
    Se una riga non è in formato JSON, viene loggato l'errore.
    """
    examples = []
    for response in responses:
        if response:
            lines = response.split("\n")
            for line in lines:
                line = line.strip()
                if line:
                    try:
                        obj = json.loads(line)
                        examples.append(obj)
                    except Exception as e:
                        print(f"Errore di parsing JSON: {e} in line: {line}")
    return examples

def save_examples_to_jsonl(examples, output_file):
    """Salva gli esempi in un file JSONL (un oggetto JSON per riga)."""
    with open(output_file, "w", encoding="utf-8") as f:
        for example in examples:
            f.write(json.dumps(example, ensure_ascii=False) + "\n")

async def main():

    context = read_context(input_file)
    print("Context letto correttamente.")

    prompt_with_context = prompt.replace("{{context}}", context)

    responses = await process_batches(prompt_with_context, total_examples, num_requests)

    examples = parse_responses(responses)
    save_examples_to_jsonl(examples, output_file)

    print(f"File JSONL creato con {len(examples)} esempi: {output_file}")
    print("="*50)

# Esegue lo script
await main()

Context letto correttamente.
{"messages": [{"role": "system", "content": "L'assistente è un chatbot esperto del Festival di Sanremo che fornisce risposte formali e dettagliate."}, {"role": "user", "content": "Quando si è svolto il Festival di Sanremo 2024?"}, {"role": "assistant", "content": "Il settantaquattresimo Festival di Sanremo si è svolto dal 6 al 10 febbraio 2024."}]}
{"messages": [{"role": "system", "content": "L'assistente è un chatbot esperto del Festival di Sanremo che fornisce risposte formali e dettagliate."}, {"role": "user", "content": "Qual è stata la durata del Festival di Sanremo 2024?"}, {"role": "assistant", "content": "Il settantaquattresimo Festival di Sanremo si è svolto dal 6 al 10 febbraio 2024, per un totale di cinque serate."}]}
{"messages": [{"role": "system", "content": "L'assistente è un chatbot esperto del Festival di Sanremo che fornisce risposte formali e dettagliate."}, {"role": "user", "content": "Chi ha condotto il Festival di Sanremo 2024?"}, {"ro

In [12]:
# Mostrare il dataset

with open(output_file, "r", encoding="utf-8") as f:
        print(f.read())

{"messages": [{"role": "system", "content": "L'assistente è un chatbot esperto del Festival di Sanremo che fornisce risposte formali e dettagliate."}, {"role": "user", "content": "Chi ha condotto il Festival di Sanremo 2024?"}, {"role": "assistant", "content": "Il Festival di Sanremo 2024 è stato condotto da Amadeus, che ha ricoperto anche il ruolo di direttore artistico. Amadeus è stato affiancato da vari co-conduttori tra cui Marco Mengoni, Giorgia, Teresa Mannino, Lorella Cuccarini e Fiorello."}]}
{"messages": [{"role": "system", "content": "L'assistente è un chatbot esperto del Festival di Sanremo che fornisce risposte formali e dettagliate."}, {"role": "user", "content": "Quali sono le date del Festival di Sanremo 2024?"}, {"role": "assistant", "content": "Il settantaquattresimo Festival di Sanremo si è svolto dal 6 al 10 febbraio 2024."}]}
{"messages": [{"role": "system", "content": "L'assistente è un chatbot esperto del Festival di Sanremo che fornisce risposte formali e dettagl

# Dataset validation

In [19]:
data_path = output_file

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

# Format error checks
format_errors = {}

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        function_call = message.get("function_call", None)

        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

Num examples: 5
First example:
{'role': 'system', 'content': "L'assistente è un chatbot esperto del Festival di Sanremo che fornisce risposte formali e dettagliate."}
{'role': 'user', 'content': 'Chi ha condotto il Festival di Sanremo 2024?'}
{'role': 'assistant', 'content': "Il Festival di Sanremo 2024 è stato condotto da Amadeus, che ha ricoperto questo ruolo per il quinto anno consecutivo, con l'affiancamento di vari co-conduttori tra cui Marco Mengoni, Giorgia, Teresa Mannino, Lorella Cuccarini e Fiorello."}
No errors found


# Cost estimation

In [ ]:
import tiktoken
import numpy as np
from collections import defaultdict

encoding = tiktoken.get_encoding("cl100k_base")
encoding = tiktoken.encoding_for_model(openai_model)

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

# print("Num examples missing system message:", n_missing_system)
# print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
# print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 65536 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 65,536 token limit, they will be truncated during fine-tuning")

# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 65536

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training\n")
print(f"By default, you'll train for {n_epochs} epochs on this dataset\n")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

###
#  Check https://platform.openai.com/docs/pricing for the most up-to-date pricing information

# Estimate costs
print("https://platform.openai.com/docs/pricing")
gpt_4o_mini_pricing = 3.00 / 1000000  # $3.00 per million tokens
cost = (n_epochs * n_billing_tokens_in_dataset) * gpt_4o_mini_pricing

print(f"\n\nCost estimation ~ USD {cost:.3f}")


#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 80, 120
mean / median: 93.0, 88.0
p5 / p95: 82.8, 108.0

0 examples may be over the 65,536 token limit, they will be truncated during fine-tuning
Dataset has ~465 tokens that will be charged for during training
By default, you'll train for 20 epochs on this dataset
By default, you'll be charged for ~9300 tokens
https://platform.openai.com/docs/pricing


Cost estimation ~ USD 0.028
